### Project Week 05
名企BI班 week05 谢雅楠 20200926

#### Thinking

1. 在CTR点击率预估中，使用GBDT+LR的原理是什么?  
GBDT+LR是具有stacking思想的二分类器模型，用来解决二分类问题。  
GBDT将特征进行组合，然后传入给线性分类器；LR对GBDT产生的输入数据进行分类。

2. Wide & Deep的模型结构是怎样的，为什么能通过具备记忆和泛化能力（memorization and generalization）？  
memorization，记忆能力，学习items或者features之间的相关频率，在历史数据中探索相关性的可行性；generalization，泛化（推理）能力，基于相关性的传递，去探索一些在过去没有出现过的特征组合。结合线性模型的记忆能力和DNN模型的泛化能力，在训练过程中同时优化两个模型的参数。Joint Training，同时训练Wide模型和Deep模型，并将两个模型的结果的加权作为最终的预测结果。

3. 在CTR预估中，使用FM与DNN结合的方式，有哪些结合的方式，代表模型有哪些？  
1 DeepFM -- DeepFM是将Wide & Deep模型中的Wide替换成了FM模型，是并行结构，FM和DNN分开计算。  
2 NFM -- NFM算法是对embedding直接采用对位相乘（element-wise）后相加起来作为交叉特征，然后通过DNN直接将特征压缩，最后concatenate linear部分和deep部分的特征，是串行架构，将FM的结果作为DNN的输入。

4. GBDT和随机森林都是基于树的算法，它们有什么区别？  
1 随机森林使用数据的随机样本独立地训练每棵树，这种随机性有助于使模型比单个决策树更强大，并且不太可能过度拟合训练数据。    
GBDT一次构建一棵树，其中每棵新树的目标是减小先前受过训练的树的残差。（ref https://medium.com/@aravanshad/gradient-boosting-versus-random-forest-cfa3fa8f0d80）  
2 随即森林属于Bagging方法，通过自助采样的方法生成众多并行式的分类器，通过“少数服从多数”的原则来确定最终的结果。  
GBDT属于Boosting方法，通过将弱学习器提升为强学习器的集成方法来提高预测精度。

5. item流行度在推荐系统中有怎样的应用？  
1 根据内容的流行程度，将将榜单中热度的内容推荐给用户（微博热搜，TopN商品）；  
2 解决冷启动问题，根据流行度来推荐商品的算法，也就是什么内容吸引用户，就给用户推荐什么内容。

#### Action
使用Wide&Deep模型对movielens进行评分预测

In [2]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr.models import WDL
# from deepctr.inputs import SparseFeat,get_feature_names
# ref. https://blog.csdn.net/candy134834/article/details/107861246
from deepctr.feature_column import SparseFeat,get_feature_names


data = pd.read_csv("movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用WDL进行训练
model = WDL(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用WDL进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


1/1 [==============================] - 0s 232ms/step - loss: 14.1405 - mse: 14.1405 - val_loss: 14.2987 - val_mse: 14.2987
test RMSE 3.717674003997661
